In [97]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
from shapely.geometry import Point
from pyproj import CRS
from keplergl import KeplerGl
from datetime import datetime, timedelta

### ***1. Layer creation with Geopandas***

In [98]:
# Reading file
fp = r'data/bird_tracking_data.csv'
data = pd.read_csv(fp)

# Creating a geometry column as Point geometry
data['geometry'] = [Point(long, lat) for long, lat in zip(data['longitude'].to_list(), data['latitude'].to_list())]

# Creating a Geodataframe. Be aware it is CRS 4326 WGS84
geodata = gpd.GeoDataFrame(data, crs = CRS.from_epsg('4326'))

In [99]:
# Create timestamp with Pandas datetime. Inclue None as timezone
geodata['t'] = pd.to_datetime(geodata['date_time']).dt.tz_localize(None)

# Set timestamp as Index
geodata = geodata.set_index('t')

geodata.head()

,index,altitude,date_time,device_info_serial,direction,latitude,longitude,speed_2d,bird_name,geometry
t,,,,,,,,,,
2013-08-15 00:18:08,0,71,2013-08-15 00:18:08+00,851,-150.469753,49.419859,2.120733,0.150000,Eric,POINT (2.12073 49.41986)
2013-08-15 00:48:07,1,68,2013-08-15 00:48:07+00,851,-136.151141,49.419880,2.120746,2.438360,Eric,POINT (2.12075 49.41988)
2013-08-15 01:17:58,2,68,2013-08-15 01:17:58+00,851,160.797477,49.420310,2.120885,0.596657,Eric,POINT (2.12088 49.42031)
2013-08-15 01:47:51,3,73,2013-08-15 01:47:51+00,851,32.769360,49.420359,2.120859,0.310161,Eric,POINT (2.12086 49.42036)
2013-08-15 02:17:42,4,69,2013-08-15 02:17:42+00,851,45.191230,49.420331,2.120887,0.193132,Eric,POINT (2.12089 49.42033)


### ***2. Stop detection with Movingpandas***

In [100]:
# Create a Trajectory Collection with Movingpandas
traj_collection = mpd.TrajectoryCollection(geodata, 'bird_name')

In [101]:
# Define parameters in Hours and Search radio in meters
Hours = 12
SearchRadio = 1000

# stop detection
stops = mpd.TrajectoryStopDetector(traj_collection).get_stop_segments(min_duration=timedelta(hours=Hours), max_diameter=SearchRadio)

In [102]:
# Create a new Geodataframe and define geometry column
stops_start = gpd.GeoDataFrame(columns = ['geometry'])
stops_start = stops_start.set_geometry('geometry')

# Add the ID of each stop track and define it as index
stops_start['stop_id'] = [track.id for track in stops.trajectories]
stops_start= stops_start.set_index('stop_id')

# Iteration over the Stop Trajectories
for stoptrack in stops.trajectories:
    
    # add stop duration in hours
    stops_start.at[stoptrack.id,'duration_h'] =stoptrack.get_duration().seconds/3600
    
    # add length
    stops_start.at[stoptrack.id, 'length_m']=stoptrack.get_length()
    
    # add bird name
    stops_start.at[stoptrack.id, 'bird']=stoptrack.id.split('_')[0]
    
    # add datetime
    stops_start.at[stoptrack.id, 'datetime']=stoptrack.id.split('_')[1]
    
    # geometry with start point
    stops_start.at[stoptrack.id, 'geometry'] = stoptrack.get_start_location()
    
stops_start.head()

,geometry,duration_h,length_m,bird,datetime
stop_id,,,,,
Eric_2013-10-15 17:25:39,POINT (2.73159 50.19140),12.033056,1075.796616,Eric,2013-10-15 17:25:39
Eric_2013-10-16 17:20:43,POINT (2.73038 50.19840),12.249444,1143.566831,Eric,2013-10-16 17:20:43
Eric_2013-10-17 17:38:50,POINT (2.73382 50.19118),12.005556,631.531797,Eric,2013-10-17 17:38:50
Eric_2013-10-18 17:12:11,POINT (2.73340 50.19094),12.293889,1069.100195,Eric,2013-10-18 17:12:11
Eric_2013-10-19 17:17:12,POINT (2.73157 50.19789),12.294722,1322.449140,Eric,2013-10-19 17:17:12


In [103]:
# Reset indexes
stops_start = stops_start.reset_index(drop=True)
stops_start.head()

,geometry,duration_h,length_m,bird,datetime
0,POINT (2.73159 50.19140),12.033056,1075.796616,Eric,2013-10-15 17:25:39
1,POINT (2.73038 50.19840),12.249444,1143.566831,Eric,2013-10-16 17:20:43
2,POINT (2.73382 50.19118),12.005556,631.531797,Eric,2013-10-17 17:38:50
3,POINT (2.73340 50.19094),12.293889,1069.100195,Eric,2013-10-18 17:12:11
4,POINT (2.73157 50.19789),12.294722,1322.449140,Eric,2013-10-19 17:17:12


In [104]:
stops_start.dtypes

geometry      geometry
duration_h     float64
length_m       float64
bird            object
datetime        object
dtype: object

In [105]:
# Reset indexes
geodata= geodata.reset_index(drop=True)
geodata.head()

,index,altitude,date_time,device_info_serial,direction,latitude,longitude,speed_2d,bird_name,geometry
0,0,71,2013-08-15 00:18:08+00,851,-150.469753,49.419859,2.120733,0.150000,Eric,POINT (2.12073 49.41986)
1,1,68,2013-08-15 00:48:07+00,851,-136.151141,49.419880,2.120746,2.438360,Eric,POINT (2.12075 49.41988)
2,2,68,2013-08-15 01:17:58+00,851,160.797477,49.420310,2.120885,0.596657,Eric,POINT (2.12088 49.42031)
3,3,73,2013-08-15 01:47:51+00,851,32.769360,49.420359,2.120859,0.310161,Eric,POINT (2.12086 49.42036)
4,4,69,2013-08-15 02:17:42+00,851,45.191230,49.420331,2.120887,0.193132,Eric,POINT (2.12089 49.42033)


In [106]:
geodata.dtypes

index                    int64
altitude                 int64
date_time               object
device_info_serial       int64
direction              float64
latitude               float64
longitude              float64
speed_2d               float64
bird_name               object
geometry              geometry
dtype: object

### ***3. Point map visualization with KeplerGl***

In [107]:
# Create KeplerGl instance
m = KeplerGl(height=600)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [108]:
m.add_data(stops_start, 'stops')

In [109]:
m.add_data(geodata, 'trajectories')

In [110]:
m

KeplerGl(data={'stops': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 2…

In [120]:
config={
  "version": "v1",
  "config": {
    "visState": {
      "filters": [
        {
          "dataId": [
            "trajectories"
          ],
          "id": "6y9m1jfk",
          "name": [
            "date_time"
          ],
          "type": "timeRange",
          "value": [
            1376524868000,
            1376669706000
          ],
          "enlarged": True,
          "plotType": "histogram",
          "animationWindow": "free",
          "yAxis": None
        }
      ],
      "layers": [
        {
          "id": "9hel8en",
          "type": "geojson",
          "config": {
            "dataId": "trajectories",
            "label": "trajectories",
            "color": [
              136,
              87,
              44
            ],
            "columns": {
              "geojson": "geometry"
            },
            "isVisible": True,
            "visConfig": {
              "opacity": 0.3,
              "strokeOpacity": 0.8,
              "thickness": 0.5,
              "strokeColor": None,
              "colorRange": {
                "name": "ColorBrewer Dark2-3",
                "type": "qualitative",
                "category": "ColorBrewer",
                "colors": [
                  "#1b9e77",
                  "#d95f02",
                  "#7570b3"
                ]
              },
              "strokeColorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radius": 10,
              "sizeRange": [
                0,
                10
              ],
              "radiusRange": [
                0,
                50
              ],
              "heightRange": [
                0,
                500
              ],
              "elevationScale": 5,
              "stroked": False,
              "filled": True,
              "enable3d": False,
              "wireframe": False
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": {
              "name": "bird_name",
              "type": "string"
            },
            "colorScale": "ordinal",
            "sizeField": None,
            "sizeScale": "linear",
            "strokeColorField": None,
            "strokeColorScale": "quantile",
            "heightField": None,
            "heightScale": "linear",
            "radiusField": None,
            "radiusScale": "linear"
          }
        },
        {
          "id": "anw2j8",
          "type": "geojson",
          "config": {
            "dataId": "stops",
            "label": "stops",
            "color": [
              18,
              147,
              154
            ],
            "columns": {
              "geojson": "geometry"
            },
            "isVisible": True,
            "visConfig": {
              "opacity": 0.51,
              "strokeOpacity": 0.8,
              "thickness": 0.5,
              "strokeColor": None,
              "colorRange": {
                "name": "ColorBrewer YlOrRd-5",
                "type": "sequential",
                "category": "ColorBrewer",
                "colors": [
                  "#ffffb2",
                  "#fecc5c",
                  "#fd8d3c",
                  "#f03b20",
                  "#bd0026"
                ]
              },
              "strokeColorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radius": 10,
              "sizeRange": [
                0,
                10
              ],
              "radiusRange": [
                0,
                50
              ],
              "heightRange": [
                0,
                500
              ],
              "elevationScale": 5,
              "stroked": False,
              "filled": True,
              "enable3d": False,
              "wireframe": False
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": {
              "name": "duration_h",
              "type": "real"
            },
            "colorScale": "quantize",
            "sizeField": None,
            "sizeScale": "linear",
            "strokeColorField": None,
            "strokeColorScale": "quantile",
            "heightField": None,
            "heightScale": "linear",
            "radiusField": {
              "name": "duration_h",
              "type": "real"
            },
            "radiusScale": "sqrt"
          }
        }
      ],
      "interactionConfig": {
        "tooltip": {
          "fieldsToShow": {
            "stops": [
              {
                "name": "duration_h",
                "format": None
              },
              {
                "name": "length_m",
                "format": None
              },
              {
                "name": "bird",
                "format": None
              },
              {
                "name": "datetime",
                "format": None
              }
            ],
            "trajectories": [
              {
                "name": "index",
                "format": None
              },
              {
                "name": "altitude",
                "format": None
              },
              {
                "name": "date_time",
                "format": None
              },
              {
                "name": "device_info_serial",
                "format": None
              },
              {
                "name": "direction",
                "format": None
              }
            ]
          },
          "compareMode": False,
          "compareType": "absolute",
          "enabled": True
        },
        "brush": {
          "size": 0.5,
          "enabled": False
        },
        "geocoder": {
          "enabled": False
        },
        "coordinate": {
          "enabled": False
        }
      },
      "layerBlending": "normal",
      "splitMaps": [],
      "animationConfig": {
        "currentTime": None,
        "speed": 1
      }
    },
    "mapState": {
      "bearing": 0,
      "dragRotate": False,
      "latitude": 29.9292634399096,
      "longitude": 29.011689692537782,
      "pitch": 0,
      "zoom": 2.9999999999999973,
      "isSplit": False
    },
    "mapStyle": {
      "styleType": "dark",
      "topLayerGroups": {},
      "visibleLayerGroups": {
        "label": True,
        "road": True,
        "border": False,
        "building": True,
        "water": True,
        "land": True,
        "3d building": False
      },
      "threeDBuildingColor": [
        9.665468314072013,
        17.18305478057247,
        31.1442867897876
      ],
      "mapStyles": {}
    }
  }
}

In [121]:
m.save_to_html(file_name='index.html', config=config)

Map saved to index.html!
